# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects separate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import datetime
import deeplabcut

In [28]:
task='frontslowmo' # Enter the name of your experiment Task
experimenter='vj' # Enter the name of the experimenter
video=[r"C:\Users\vjj14\Downloads\1080p.MOV"] # Enter the paths of your videos you want to grab frames from.
time_log= open("time_log{}.txt".format(task),"w+")
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory= r'C:\Users\vjj14\Desktop\DeepLabCut',copy_videos=True) #change the working directory to where you want the folders created.

# The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
#path_config_file = '/home/Mackenzie/Reaching/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\videos"
Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\labeled-data"
Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\training-datasets"
Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\dlc-models"
Copying the videos
C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\videos\1080p.MOV
Generated "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\config.yaml"

A new project with name frontslowmo-vj-2019-06-06 is created at C:\Users\vjj14\Desktop\DeepLabCut and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project (at any stage).


In [2]:
path_config_file='C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\config.yaml'
project_name='frontslowmo'
def write_log(text, new_line=True):
    time_log= open("{0}time_log{1}.txt".format(path_config_file[:-11], project_name),"a+")
    if new_line:
        time_log.write("\n")
    time_log.write(text)
    time_log.close()
    print("log written")

In [9]:
print(path_config_file[:-11])
write_log("test1")

C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\
log written


## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [34]:
start = datetime.datetime.now()
write_log(str(start) + "***extract_frames start")

%matplotlib inline
deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 

end = datetime.datetime.now()
write_log(str(end) + "***extract_frames end")
write_log("   time elapsed:" + str(end-start))

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080p.MOV ?
yes/nono

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [40]:
time_log.close()

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [4]:
start = datetime.datetime.now()
write_log(str(start) + "***label_frames start")

%gui wx
deeplabcut.label_frames(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***label_frames end")
write_log("   time elapsed:" + str(end-start))

ERROR:root:Cannot activate multiple GUI eventloops


ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'wx' is currently running

**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [10]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by vj.
C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\labeled-data\1080p_labeled  already exists!
They are stored in the following folder: C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\labeled-data\1080p_labeled.
Attention: C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\labeled-data\1080p does not appear to have labeled data!
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [12]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\training-datasets\iteration-0\UnaugmentedDataSet_frontslowmoJun5  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [6]:
?deeplabcut.create_training_dataset

## Start training - If you want to use a CPU, continue. 
### If you want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [10]:
from time import sleep
def prompt_with_timeout(prompt_message):
  print('Waiting... please press Ctrl-C when you wish to proceed.')
  try:
    for i in range(0, 2*60): # 2 minutes
      sleep(1)
    return "No Input Received"
  except KeyboardInterrupt:
    return input(prompt_message)

In [36]:
start = datetime.datetime.now()
write_log(str(start) + "***train_network start")
write_log("   DESCRIPTION:" + prompt_with_timeout("Training description(Network, Iterations, etc) "))

deeplabcut.train_network(path_config_file, saveiters=1000, displayiters=100, maxiters=300001)

end = datetime.datetime.now()
write_log(str(end) + "***train_network end")
write_log("   time elapsed:" + str(end-start))

log written
Waiting... please press Ctrl-C when you wish to proceed.
Training description(Network, Iterations, etc) continuation of the second network (the best one), maybe include relabeled frames from clock time.
log written


Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['LeftHand', 'RightHand', 'Nose', 'Pellet'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_frontslowmoJun5\\frontslowmo_vj95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\dlc-models\\iteration-1\\frontslowmoJun5-trainset95shuffle1\\train\\snapshot-30000',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_frontslowmoJun5\\Documentation_data-frontslowmo_95shuffle1.pi

INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1\train\snapshot-30000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1\train\snapshot-30000


Max_iters overwritten as 300001
Display_iters overwritten as 100
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\dlc-models\\iteration-1\\frontslowmoJun5-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400

...

iteration: 166100 loss: 0.0013 lr: 0.02
iteration: 166200 loss: 0.0011 lr: 0.02
iteration: 166300 loss: 0.0011 lr: 0.02
iteration: 166400 loss: 0.0010 lr: 0.02
iteration: 166500 loss: 0.0012 lr: 0.02
iteration: 166600 loss: 0.0013 lr: 0.02
iteration: 166700 loss: 0.0014 lr: 0.02
iteration: 166800 loss: 0.0012 lr: 0.02
iteration: 166900 loss: 0.0014 lr: 0.02
iteration: 167000 loss: 0.0012 lr: 0.02
iteration: 167100 loss: 0.0015 lr: 0.02
iteration: 167200 loss: 0.0013 lr: 0.02
iteration: 167300 loss: 0.0013 lr: 0.02
iteration: 167400 loss: 0.0014 lr: 0.02
iteration: 167500 loss: 0.0013 lr: 0.02
iteration: 167600 loss: 0.0012 lr: 0.02
iteration: 167700 loss: 0.0013 lr: 0.02
iteration: 167800 loss: 0.0013 lr: 0.02
iteration: 167900 loss: 0.0013 lr: 0.02
iteration: 168000 loss: 0.0013 lr: 0.02
iteration: 168100 loss: 0.0014 lr: 0.02
iteration: 168200 loss: 0.0013 lr: 0.02
iteration: 168300 loss: 0.0011 lr: 0.02
iteration: 168400 loss: 0.0013 lr: 0.02
iteration: 168500 loss: 0.0014 lr: 0.02


iteration: 206900 loss: 0.0012 lr: 0.02
iteration: 207000 loss: 0.0013 lr: 0.02
iteration: 207100 loss: 0.0012 lr: 0.02
iteration: 207200 loss: 0.0013 lr: 0.02
iteration: 207300 loss: 0.0012 lr: 0.02
iteration: 207400 loss: 0.0011 lr: 0.02
iteration: 207500 loss: 0.0014 lr: 0.02
iteration: 207600 loss: 0.0014 lr: 0.02
iteration: 207700 loss: 0.0014 lr: 0.02
iteration: 207800 loss: 0.0011 lr: 0.02
iteration: 207900 loss: 0.0012 lr: 0.02
iteration: 208000 loss: 0.0013 lr: 0.02
iteration: 208100 loss: 0.0013 lr: 0.02
iteration: 208200 loss: 0.0012 lr: 0.02
iteration: 208300 loss: 0.0014 lr: 0.02
iteration: 208400 loss: 0.0012 lr: 0.02
iteration: 208500 loss: 0.0015 lr: 0.02
iteration: 208600 loss: 0.0012 lr: 0.02
iteration: 208700 loss: 0.0011 lr: 0.02
iteration: 208800 loss: 0.0013 lr: 0.02
iteration: 208900 loss: 0.0012 lr: 0.02
iteration: 209000 loss: 0.0011 lr: 0.02
iteration: 209100 loss: 0.0012 lr: 0.02
iteration: 209200 loss: 0.0012 lr: 0.02
iteration: 209300 loss: 0.0013 lr: 0.02


iteration: 247700 loss: 0.0012 lr: 0.02
iteration: 247800 loss: 0.0011 lr: 0.02
iteration: 247900 loss: 0.0013 lr: 0.02
iteration: 248000 loss: 0.0011 lr: 0.02
iteration: 248100 loss: 0.0011 lr: 0.02
iteration: 248200 loss: 0.0013 lr: 0.02
iteration: 248300 loss: 0.0011 lr: 0.02
iteration: 248400 loss: 0.0011 lr: 0.02
iteration: 248500 loss: 0.0012 lr: 0.02
iteration: 248600 loss: 0.0012 lr: 0.02
iteration: 248700 loss: 0.0013 lr: 0.02
iteration: 248800 loss: 0.0011 lr: 0.02
iteration: 248900 loss: 0.0011 lr: 0.02
iteration: 249000 loss: 0.0011 lr: 0.02
iteration: 249100 loss: 0.0012 lr: 0.02
iteration: 249200 loss: 0.0012 lr: 0.02
iteration: 249300 loss: 0.0011 lr: 0.02
iteration: 249400 loss: 0.0012 lr: 0.02
iteration: 249500 loss: 0.0013 lr: 0.02
iteration: 249600 loss: 0.0012 lr: 0.02
iteration: 249700 loss: 0.0013 lr: 0.02
iteration: 249800 loss: 0.0011 lr: 0.02
iteration: 249900 loss: 0.0013 lr: 0.02
iteration: 250000 loss: 0.0011 lr: 0.02
iteration: 250100 loss: 0.0011 lr: 0.02


iteration: 288500 loss: 0.0011 lr: 0.02
iteration: 288600 loss: 0.0010 lr: 0.02
iteration: 288700 loss: 0.0010 lr: 0.02
iteration: 288800 loss: 0.0013 lr: 0.02
iteration: 288900 loss: 0.0012 lr: 0.02
iteration: 289000 loss: 0.0011 lr: 0.02
iteration: 289100 loss: 0.0012 lr: 0.02
iteration: 289200 loss: 0.0012 lr: 0.02
iteration: 289300 loss: 0.0013 lr: 0.02
iteration: 289400 loss: 0.0013 lr: 0.02
iteration: 289500 loss: 0.0012 lr: 0.02
iteration: 289600 loss: 0.0011 lr: 0.02
iteration: 289700 loss: 0.0011 lr: 0.02
iteration: 289800 loss: 0.0011 lr: 0.02
iteration: 289900 loss: 0.0013 lr: 0.02
iteration: 290000 loss: 0.0011 lr: 0.02
iteration: 290100 loss: 0.0010 lr: 0.02
iteration: 290200 loss: 0.0011 lr: 0.02
iteration: 290300 loss: 0.0011 lr: 0.02
iteration: 290400 loss: 0.0011 lr: 0.02
iteration: 290500 loss: 0.0011 lr: 0.02
iteration: 290600 loss: 0.0011 lr: 0.02
iteration: 290700 loss: 0.0011 lr: 0.02
iteration: 290800 loss: 0.0013 lr: 0.02
iteration: 290900 loss: 0.0011 lr: 0.02


The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.
log written
log written


In [43]:
#iter0  ~ 1 hour and 20 minutes
#iter1 start-1:07pm

In [18]:
?deeplabcut.train_network

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [39]:
start = datetime.datetime.now()
write_log(str(start) + "***evaluate_network start")

deeplabcut.evaluate_network(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***evalaute_network end")
write_log("   time elapsed:" + str(end-start))

log written
C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05/evaluation-results/  already exists!


Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['LeftHand', 'RightHand', 'Nose', 'Pellet'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_frontslowmoJun5\\frontslowmo_vj95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_frontslowmoJun5\\Documentation_data-frontslowmo_95shuffle1.pickl

C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\evaluation-results\iteration-1\frontslowmoJun5-trainset95shuffle1  already exists!
Running  DeepCut_resnet50_frontslowmoJun5shuffle1_297000  with # of trainingiterations: 297000
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1\train\snapshot-297000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1\train\snapshot-297000


Analyzing data...


204it [00:37,  5.49it/s]


Done and results stored for snapshot:  snapshot-297000
Results for 297000  training iterations: 95 1 train error: 1.72 pixels. Test error: 21.37  pixels.
With pcutoff of 0.1  train error: 1.72 pixels. Test error: 13.19 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)
log written
log written


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [21]:
# videofile_path = [r"C:\Users\vjj14\Downloads\compressed25.mp4"] #Enter the list of videos to analyze.
# videos=[[r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed50.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed30.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed27.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed25.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed20.mp4"]]
videos=[[r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\gp.mp4"]]
for videofile_path in videos:
    start = datetime.datetime.now()
    write_log(str(start) + "***analyze_videos start")
    write_log("   VIDEOS: " + str(videofile_path))

    deeplabcut.analyze_videos(path_config_file,videofile_path, save_as_csv=True, cropping=[1050, 1700, 250, 650])

    end = datetime.datetime.now()
    write_log(str(end) + "***analyze_videos end")
    write_log("   time elapsed:" + str(end-start))

log written
log written
Overwriting cropping parameters: [1050, 1700, 250, 650]
These are used for all videos, but won't be save to the cfg file.
Using snapshot-28000 for model C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-0\frontslowmoJun5-trainset95shuffle1
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-0\frontslowmoJun5-trainset95shuffle1\train\snapshot-28000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-0\frontslowmoJun5-trainset95shuffle1\train\snapshot-28000


Starting to analyze %  C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\gp.mp4
Loading  C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\gp.mp4
Duration of video [s]:  200.3 , recorded with  119.88 fps!
Overall # of frames:  24012  found with (before cropping) frame dimensions:  2704 1520
Starting to extract posture
Cropping based on the x1 = 1050 x2 = 1700 y1 = 250 y2 = 650. You can adjust the cropping coordinates in the config.yaml file.






  0%|                                                                                        | 0/24012 [00:00<?, ?it/s]



  2%|█▌                                                                            | 480/24012 [00:07<06:17, 62.39it/s]



  3%|██▎                                                                           | 720/24012 [00:14<07:47, 49.80it/s]



  4%|███                                                                           | 960/24012 [00:21<08:40, 44.30it/s]



  5%|███▊                                                                         | 1200/24012 [00:28<09:24, 40.40it/s]



  6%|████▌                                                                        | 1440/24012 [00:35<09:50, 38.23it/s]



  7%|█████▍                                                                       | 1680/24012 [00:42<10:03, 36.98it/s]



  8%|██████▏                                                                      | 1920/24012 [00:49<10:11, 36.14it/s]



  9%|██████▉

Detected frames:  23995
Saving results in C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!
log written
log written


In [ ]:
#Keep Track of time for analyzing videos
#Original: 940 mb
#25:134 mb, took 20:55

In [6]:
?deeplabcut.analyze_videos

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [8]:
alg = 'uncertain'
param = 0.0002 #epsilon or p_bound

start = datetime.datetime.now()
write_log(str(start) + "***extract_outlier_frames start")

deeplabcut.extract_outlier_frames(path_config_file, videofile_path, outlieralgorithm=alg, p_bound=param)
num_frames = prompt_with_timeout("number of frames selected?")
write_log("   VIDEOS: {0}, ALGORITHM: {1}, PARAMETER: {2}, NUM FRAMES: {3}".format(str(videofile_path), alg, param, num_frames))

end = datetime.datetime.now()
write_log(str(end) + "***analyze_videos end")
write_log("   time elapsed:" + str(end-start))

network parameters: DeepCut_resnet50_frontslowmoJun5shuffle1_28000
Method  uncertain  found  12  putative outlier frames.
Do you want to proceed with extracting  200  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, p_bound, comparisonbodyparts) or use a different method.
yes/noyes
Frames from video 1080p  already extracted (more will be added)!
Loading video...
Duration of video [s]:  518.6666666666666 , recorded @  30.0 fps!
Overall # of frames:  15560 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 518.7  seconds.
Let's select frames indices: [113, 154, 155, 156, 157, 158, 160, 161, 162, 13116, 13117, 13126]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\1080p.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.

In [3]:
videofile_path = [r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080p.MOV"]
videofile_path

['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\videos\\1080p.MOV']

In [38]:
?deeplabcut.extract_outlier_frames

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [4]:
start = datetime.datetime.now()
write_log(str(start) + "***refine_frames start")

%gui wx
deeplabcut.refine_labels(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***refine_frames end")
write_log("   time elapsed:" + str(end-start))


ERROR:root:Cannot activate multiple GUI eventloops


Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [5]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
start = datetime.datetime.now()
write_log(str(start) + "***refine_frames start")

deeplabcut.merge_datasets(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***refine_frames end")
write_log("   time elapsed:" + str(end-start))

Merged data sets and updated refinement iteration to 2.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [6]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
# videos=[[r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed50.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed30.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed27.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed25.mp4"], [r"C:\Users\vjj14\Downloads\frontslowmoCOMPRESSED\compressed20.mp4"]]
# videos=[[r"C:\Users\vjj14\Desktop\DeepLabCut\paperfrontslowmo-vj-2019-06-10\videos\paperfrontslowmo_Trim.mp4"]]
for videofile_path in videos:
    start = datetime.datetime.now()
    write_log(str(start) + "***create_labeled_video start")
    write_log("   VIDEOS: {}".format(str(videofile_path)))

    deeplabcut.create_labeled_video(path_config_file,videofile_path, save_frames=True)

    end = datetime.datetime.now()
    write_log(str(end) + "***create_labeled_video end")
    write_log("   time elapsed:" + str(end-start))

log written
log written
Starting %  C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos ['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\videos\\gp.mp4']
Loading  C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\gp.mp4 and data.
True 1050 1700 250 650
24012
Duration of video [s]:  200.16 , recorded with  119.88 fps!
Overall # of frames:  23995 with cropped frame dimensions:  650 400
Generating frames and creating video.






  0%|                                                                                        | 0/23995 [00:00<?, ?it/s]



  0%|                                                                              | 1/23995 [00:00<1:58:24,  3.38it/s]



  0%|                                                                              | 2/23995 [00:00<2:04:09,  3.22it/s]



  0%|                                                                              | 3/23995 [00:00<2:03:23,  3.24it/s]



  0%|                                                                              | 4/23995 [00:01<2:05:15,  3.19it/s]



  0%|                                                                              | 5/23995 [00:01<2:02:57,  3.25it/s]



  0%|                                                                              | 6/23995 [00:01<2:03:45,  3.23it/s]



  0%|                                                                              | 7/23995 [00:02<2:02:23,  3.27it/s]



  0%|       

  1%|▍                                                                           | 132/23995 [00:42<2:11:38,  3.02it/s]



  1%|▍                                                                           | 133/23995 [00:43<2:08:05,  3.10it/s]



  1%|▍                                                                           | 134/23995 [00:43<2:06:09,  3.15it/s]



  1%|▍                                                                           | 135/23995 [00:43<2:05:25,  3.17it/s]



  1%|▍                                                                           | 136/23995 [00:44<2:03:43,  3.21it/s]



  1%|▍                                                                           | 137/23995 [00:44<2:02:35,  3.24it/s]



  1%|▍                                                                           | 138/23995 [00:44<2:01:51,  3.26it/s]



  1%|▍                                                                           | 139/23995 [00:45<2:01:20,  3.28it/s]



  1%|▍          

  1%|▊                                                                           | 264/23995 [01:25<2:08:41,  3.07it/s]



  1%|▊                                                                           | 265/23995 [01:25<2:05:28,  3.15it/s]



  1%|▊                                                                           | 266/23995 [01:25<2:05:27,  3.15it/s]



  1%|▊                                                                           | 267/23995 [01:26<2:15:01,  2.93it/s]



  1%|▊                                                                           | 268/23995 [01:26<2:19:42,  2.83it/s]



  1%|▊                                                                           | 269/23995 [01:26<2:13:45,  2.96it/s]



  1%|▊                                                                           | 270/23995 [01:27<2:08:39,  3.07it/s]



  1%|▊                                                                           | 271/23995 [01:27<2:05:18,  3.16it/s]



  1%|▊          

  2%|█▎                                                                          | 396/23995 [02:07<2:03:56,  3.17it/s]



  2%|█▎                                                                          | 397/23995 [02:07<2:01:56,  3.23it/s]



  2%|█▎                                                                          | 398/23995 [02:07<2:00:39,  3.26it/s]



  2%|█▎                                                                          | 399/23995 [02:07<1:59:59,  3.28it/s]



  2%|█▎                                                                          | 400/23995 [02:08<1:59:31,  3.29it/s]



  2%|█▎                                                                          | 401/23995 [02:08<1:59:08,  3.30it/s]



  2%|█▎                                                                          | 402/23995 [02:08<1:58:41,  3.31it/s]



  2%|█▎                                                                          | 403/23995 [02:09<1:58:50,  3.31it/s]



  2%|█▎         

...

 30%|██████████████████████▋                                                    | 7260/23995 [38:49<1:25:05,  3.28it/s]



 30%|██████████████████████▋                                                    | 7261/23995 [38:49<1:25:12,  3.27it/s]



 30%|██████████████████████▋                                                    | 7262/23995 [38:49<1:24:43,  3.29it/s]



 30%|██████████████████████▋                                                    | 7263/23995 [38:49<1:24:35,  3.30it/s]



 30%|██████████████████████▋                                                    | 7264/23995 [38:50<1:23:39,  3.33it/s]



 30%|██████████████████████▋                                                    | 7265/23995 [38:50<1:23:24,  3.34it/s]



 30%|██████████████████████▋                                                    | 7266/23995 [38:50<1:23:19,  3.35it/s]



 30%|██████████████████████▋                                                    | 7267/23995 [38:51<1:23:41,  3.33it/s]



 30%|███████████

 31%|███████████████████████                                                    | 7392/23995 [39:30<1:24:04,  3.29it/s]



 31%|███████████████████████                                                    | 7393/23995 [39:30<1:23:45,  3.30it/s]



 31%|███████████████████████                                                    | 7394/23995 [39:31<1:23:18,  3.32it/s]



 31%|███████████████████████                                                    | 7395/23995 [39:31<1:25:52,  3.22it/s]



 31%|███████████████████████                                                    | 7396/23995 [39:31<1:28:15,  3.13it/s]



 31%|███████████████████████                                                    | 7397/23995 [39:31<1:27:06,  3.18it/s]



 31%|███████████████████████                                                    | 7398/23995 [39:32<1:25:37,  3.23it/s]



 31%|███████████████████████▏                                                   | 7399/23995 [39:32<1:24:55,  3.26it/s]



 31%|███████████

 31%|███████████████████████▌                                                   | 7524/23995 [40:11<1:21:54,  3.35it/s]



 31%|███████████████████████▌                                                   | 7525/23995 [40:11<1:23:02,  3.31it/s]



 31%|███████████████████████▌                                                   | 7526/23995 [40:12<1:28:25,  3.10it/s]



 31%|███████████████████████▌                                                   | 7527/23995 [40:12<1:26:12,  3.18it/s]



 31%|███████████████████████▌                                                   | 7528/23995 [40:12<1:26:56,  3.16it/s]



 31%|███████████████████████▌                                                   | 7529/23995 [40:13<1:25:24,  3.21it/s]



 31%|███████████████████████▌                                                   | 7530/23995 [40:13<1:26:37,  3.17it/s]



 31%|███████████████████████▌                                                   | 7531/23995 [40:13<1:30:09,  3.04it/s]



 31%|███████████

 32%|███████████████████████▉                                                   | 7656/23995 [40:53<1:24:48,  3.21it/s]



 32%|███████████████████████▉                                                   | 7657/23995 [40:53<1:23:53,  3.25it/s]



 32%|███████████████████████▉                                                   | 7658/23995 [40:53<1:22:54,  3.28it/s]



 32%|███████████████████████▉                                                   | 7659/23995 [40:54<1:23:07,  3.28it/s]



 32%|███████████████████████▉                                                   | 7660/23995 [40:54<1:22:32,  3.30it/s]



 32%|███████████████████████▉                                                   | 7661/23995 [40:54<1:23:11,  3.27it/s]



 32%|███████████████████████▉                                                   | 7662/23995 [40:55<1:22:44,  3.29it/s]



 32%|███████████████████████▉                                                   | 7663/23995 [40:55<1:22:20,  3.31it/s]



 32%|███████████

 32%|████████████████████████▎                                                  | 7788/23995 [41:34<1:21:55,  3.30it/s]



 32%|████████████████████████▎                                                  | 7789/23995 [41:34<1:21:25,  3.32it/s]



 32%|████████████████████████▎                                                  | 7790/23995 [41:34<1:22:17,  3.28it/s]



 32%|████████████████████████▎                                                  | 7791/23995 [41:35<1:22:19,  3.28it/s]

In [22]:
?deeplabcut.create_labeled_video

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [45]:
videofile_path

['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\videos\\1080p.MOV']

In [ ]:
%matplotlib notebook 
#for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [58]:
?deeplabcut.plot_trajectories

In [68]:
path_config_file

'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\config.yaml'

In [26]:
import numpy as np
import pandas as pd

In [27]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080pDeepCut_resnet50_frontslowmoJun5shuffle1_30000.h5")
df.describe()

scorer    DeepCut_resnet50_frontslowmoJun5shuffle1_30000                \
bodyparts                                       LeftHand                 
coords                                                 x             y   
count                                       15560.000000  15560.000000   
mean                                          971.243753    725.853752   
std                                           171.896374    139.390499   
min                                            -3.870381      0.415488   
25%                                           914.894261    649.265245   
50%                                          1024.541650    783.569860   
75%                                          1064.598903    821.210082   
max                                          1611.307688   1088.421771   

scorer                                                             \
bodyparts                   RightHand                               
coords       likelihood             x             y    likelihood   
count      15560.000000  15560.000000  15560.000000  15560.000000   
mean           0.699717    906.089441    738.766505      0.503567   
std            0.357561    156.673818    166.830793      0.400728   
min            0.000027    319.477097      1.081914      0.000040   
25%            0.397388    850.580156    651.611459      0.060324   
50%            0.910440    905.468754    778.514918      0.466857   
75%            0.986921    940.134475    858.174915      0.949224   
max            0.999329   1609.412701   1092.391703      0.999041   

scorer                                                             \
bodyparts          Nose                                    Pellet   
coords                x             y    likelihood             x   
count      15560.000000  15560.000000  15560.000000  15560.000000   
mean         931.766309    656.488581      0.671245    957.103689   
std          169.306891    191.976523      0.363396    159.386299   
min          298.748627     -0.059892      0.000104     -5.040360   
25%          909.805872    595.394240      0.306699    922.235798   
50%          918.396317    703.317179      0.875269    939.903313   
75%          928.343641    794.728728      0.960582    953.739810   
max         1623.930874   1088.734671      0.998151   1613.409633   

scorer                                 
bodyparts                              
coords                y    likelihood  
count      15560.000000  15560.000000  
mean         811.306070      0.322458  
std          177.523694      0.418997  
min            0.012989      0.000098  
25%          671.503496      0.027885  
50%          847.006353      0.048265  
75%          940.497040      0.954270  
max         1089.468144      0.997180

In [15]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080pDeepCut_resnet50_frontslowmoJun5shuffle1_30000.h5")
df.describe()

scorer    DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000  \
bodyparts                                                     LeftHand   
coords                                                               x   
count                                             612.000000             
mean                                             1294.587569             
std                                               566.489027             
min                                                -0.173059             
25%                                               955.338525             
50%                                              1458.549845             
75%                                              1644.293059             
max                                              2161.044888             

scorer                                                                    \
bodyparts                             RightHand                            
coords               y  likelihood            x            y  likelihood   
count       612.000000  612.000000   612.000000   612.000000  612.000000   
mean       2455.822662    0.042246   609.627308  2811.817150    0.078364   
std         890.589556    0.138305   523.969844   763.555947    0.202453   
min           1.988913    0.000032     0.665848    76.401595    0.000153   
25%        1989.080632    0.000861     3.883312  2168.396138    0.001087   
50%        2089.116435    0.002397   764.576838  2395.076544    0.003136   
75%        3421.296761    0.011128   970.543810  3556.132724    0.019450   
max        3840.645847    0.977641  2139.572770  3840.410774    0.992057   

scorer                                                                     \
bodyparts         Nose                                Pellet                
coords               x            y  likelihood            x            y   
count       612.000000   612.000000  612.000000   612.000000   612.000000   
mean       1000.140640  2138.825481    0.222379  1161.420468  3607.459208   
std         249.147238   615.631056    0.313997   635.214989   435.197877   
min           0.269251   865.347540    0.000097     2.184450  1940.036492   
25%         990.724893  1820.097459    0.004228   736.731850  3637.395378   
50%        1030.763698  1986.446780    0.050586   963.540944  3822.212681   
75%        1069.846140  2166.098999    0.300523  1822.975545  3837.997508   
max        2152.967607  3841.093842    0.998524  2158.995823  3841.865475   

scorer                 
bodyparts              
coords     likelihood  
count      612.000000  
mean         0.364173  
std          0.308807  
min          0.000428  
25%          0.070970  
50%          0.276980  
75%          0.658898  
max          0.989263

In [71]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\front4k60fpszoom3.0_Trim-nv-2019-06-03\videos\front4k60fpszoom3.0_Trim2DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000.h5")
df.describe()

scorer    DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000  \
bodyparts                                                     LeftHand   
coords                                                               x   
count                                             396.000000             
mean                                             1081.024990             
std                                               801.865156             
min                                                -1.075624             
25%                                               306.234073             
50%                                              1099.686393             
75%                                              1978.196430             
max                                              2162.322889             

scorer                                                                    \
bodyparts                             RightHand                            
coords               y  likelihood            x            y  likelihood   
count       396.000000  396.000000   396.000000   396.000000  396.000000   
mean       1915.892243    0.028620   507.260805  2028.133410    0.074291   
std         925.669503    0.089883   570.154718   789.584933    0.167471   
min           4.104054    0.000054     1.491787     0.110741    0.000132   
25%        1363.246884    0.001314     6.579469  1460.181150    0.001657   
50%        1541.223487    0.004285   137.076892  1945.901693    0.008166   
75%        2062.913943    0.013648   941.707332  2366.580038    0.052594   
max        3840.759733    0.904895  2135.639879  3840.026480    0.958066   

scorer                                                                     \
bodyparts         Nose                                Pellet                
coords               x            y  likelihood            x            y   
count       396.000000   396.000000  396.000000   396.000000   396.000000   
mean        832.015528  2042.639285    0.122190  1359.938273  3524.089973   
std         477.821564   905.199187    0.237828   693.038635   567.594545   
min          -1.233883     1.529313    0.000029     1.956876  1154.500621   
25%         795.323176  1530.546586    0.000845   649.109436  3461.410198   
50%         980.203261  1763.745589    0.006032  1651.501211  3830.206299   
75%        1059.898767  2137.143606    0.097552  1995.377958  3838.244082   
max        2155.672404  3839.283205    0.997360  2159.782144  3843.716391   

scorer                 
bodyparts              
coords     likelihood  
count      396.000000  
mean         0.122058  
std          0.173181  
min          0.000364  
25%          0.009609  
50%          0.044030  
75%          0.171100  
max          0.900990

In [76]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\front4k60fpszoom3.0_Trim-nv-2019-06-03\videos\front1080p240fpszoom1.0DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000.h5")
df.describe()

scorer    DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000  \
bodyparts                                                     LeftHand   
coords                                                               x   
count                                           15512.000000             
mean                                              476.662807             
std                                               199.051516             
min                                                -3.072265             
25%                                               556.934558             
50%                                               571.585048             
75%                                               572.115142             
max                                               576.400276             

scorer                                                             \
bodyparts                                 RightHand                 
coords                y    likelihood             x             y   
count      15512.000000  15512.000000  15512.000000  15512.000000   
mean          56.168319      0.000372     94.257165    269.637234   
std          113.435357      0.000808    199.582534    103.977182   
min           -0.002832      0.000023     -1.198687     -0.875000   
25%            1.973024      0.000192      4.560107    284.634581   
50%            2.359020      0.000286      5.130947    317.105902   
75%            4.928002      0.000423      5.801471    317.801870   
max          322.130518      0.046496    573.605843    321.000363   

scorer                                                             \
bodyparts                        Nose                               
coords       likelihood             x             y    likelihood   
count      15512.000000  15512.000000  15512.000000  15512.000000   
mean           0.001873    503.875873     39.304025      0.000522   
std            0.002485    172.544028     96.968325      0.004167   
min            0.000019     -1.059059     -0.550313      0.000017   
25%            0.000632    571.378365      0.997769      0.000229   
50%            0.001176    571.993842      1.213735      0.000329   
75%            0.002189    572.493355      1.552477      0.000470   
max            0.067399    575.610920    321.239240      0.320831   

scorer                                               
bodyparts        Pellet                              
coords                x             y    likelihood  
count      15512.000000  15512.000000  15512.000000  
mean         459.721134     70.262810      0.001173  
std          220.905013    127.125900      0.006426  
min           -0.952420     -1.799350      0.000008  
25%          564.175327      1.154488      0.000486  
50%          571.777898      1.417860      0.000696  
75%          572.075551      1.793241      0.000972  
max          574.988145    321.820386      0.218525

In [35]:
csv = pd.read_csv(r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080pDeepCut_resnet50_frontslowmoJun5shuffle1_28000.csv")
csv.head(10)

,scorer,DeepCut_resnet50_frontslowmoJun5shuffle1_28000,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.1,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.2,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.3,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.4,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.5,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.6,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.7,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.8,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.9,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.10,DeepCut_resnet50_frontslowmoJun5shuffle1_28000.11
0,bodyparts,LeftHand,LeftHand,LeftHand,RightHand,RightHand,RightHand,Nose,Nose,Nose,Pellet,Pellet,Pellet
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,1390.192226409912,863.4161190986633,0.0021387303713709116,655.7109663486481,671.5528552532196,0.03249989077448845,858.313562631607,502.5887746810913,0.008192891255021095,780.1106552109122,385.51115894317627,0.005371846724301577
3,1,1342.6869158744812,1022.5391943454742,0.002777844201773405,655.604768037796,672.7327032089233,0.014964205212891102,858.8051793575287,503.0613145828247,0.009881128557026386,780.0567750260234,385.8125066757202,0.004218136891722679
4,2,1378.603021144867,839.2000513076782,0.04729203134775162,655.5680959224701,671.9902248382568,0.008627155795693398,1376.6142420768738,838.8206067085266,0.023861268535256386,779.368589758873,384.6133441925049,0.00545266130939126
5,3,717.1648300886154,528.1120610237122,0.003874382469803095,713.5224957466125,529.766140460968,0.013523245230317116,712.549768447876,530.312716960907,0.013139158487319946,779.9379644244909,384.9282178878784,0.008757703006267548
6,4,1342.3471715450287,1021.1889725923538,0.002227584132924676,772.2066966891289,391.2881712913513,0.004997084848582745,776.2603969573975,387.5884371101856,0.016647374257445335,781.6834099292755,385.06572008132935,0.03011847473680973
7,5,1341.7412056922913,1020.5597079992294,0.004295835737138987,764.2025130093098,382.7016522884369,0.006730221211910248,776.0733242034912,387.5614034831524,0.021196046844124794,782.169196844101,384.94352865219116,0.020823610946536064
8,6,782.3293192386627,393.09867787361145,0.0023416001349687576,763.9979656348005,382.5667326450348,0.00377332279458642,776.1697680950165,387.81131461262703,0.022181008011102676,781.9936457872391,385.16719913482666,0.02330200932919979
9,7,781.9441992044449,392.8441734313965,0.0025434144772589207,764.6089184880257,382.68700790405273,0.004340021871030331,776.1248214244843,386.51550006866455,0.016398178413510323,782.0050585269928,385.07819986343384,0.02907874435186386
